# mine a jumblecoin
first the setup:

In [2]:
import hashlib
from urllib import request
import urllib
import json

BASE_URL = 'http://jumblesale.localhost.run'


def pretty_print(data: object) -> None:
    print(json.dumps(data, indent=4))


def hash_dict(x: dict) -> str:
    string = json.dumps(x, sort_keys=True).encode()
    return hashlib.sha256(string).hexdigest()


def get_chain() -> dict:
    data = get_json_from_url('{}/chain'.format(BASE_URL))
    return data


def get_json_from_url(url: str) -> dict:
    with urllib.request.urlopen(url) as response:
        response_data = json.loads(response.read().decode('utf-8'))
        return response_data


def post_data_to_url(url: str, data: dict):
    # str.encode() will turn the data into bytes which we need for transmitting
    req = request.Request(url, data=json.dumps(data).encode(), headers={'content-type': 'application/json'})
    return request.urlopen(req)


def post_block(block: dict) -> str:
    return post_data_to_url('{}/block'.format(BASE_URL), block).read().decode('utf-8')


def get_last_block() -> dict:
    return get_json_from_url('{}/last_block'.format(BASE_URL))

## hashing a proof
we need a method to perform the hashing. `hash_proof` will give us the sha_256 hash of two numbers which will allow us to tell if our input, hashed with the last proof, produces a valid hash.

In [3]:
def hash_proof(last_proof: int, proof:int) -> str:
    """
    produce a sha_256 digest of the hash of the two numbers
    :param last_proof: the last proof in the chain
    :param proof: your generated proof
    :return: the hex digest
    """
    return hashlib \
        .sha256((str(last_proof) + str(proof)).encode()) \
        .hexdigest()

## creating a block
this method will allow us to create a block. it relies on us providing a proper implementation for the `proof_of_work` method, listed below:

In [4]:
def create_new_block(recipient: str) -> dict:
    """
    create a block to send to the chain
    :param recipient: your name
    :return: a dict containing all the data the chain needs to add a new block
    """
    last_block = get_last_block()
    previous_hash = hash_dict(last_block)
    proof = proof_of_work(last_block['proof'])
    return {
        'previous_hash': previous_hash,
        'proof': proof,
        'recipient': recipient
    }

## creating a proof of work
this is the method which needs your attention! `PROOF_OF_WORK_COST` provides the number of trailing `0`s we're looking for in the hash. the implementation of `proof_of_work` needs to keep trying hashes until it finds an `x` such that `hash_proof(x, last_proof)` gives a hash which ends in `000`.

In [20]:
PROOF_OF_WORK_COST = 3

def proof_of_work(last_proof_of_work) -> int:
    """
    return a number which, when concatenated by the last_proof_of_work,
    will hash to produce a string ending in 3 '0's

    example: if the last proof of work was 2256
    :param last_proof_of_work: the proof of work from the last block in the chain
    :return: a number, x, where hash_number(last_proof_of_work * x) ends in '000'
    """
    x = 5137
    hashed_proof = hash_proof(last_proof_of_work, x)
    # this should end in 000
    print('hash of {} and {} is {}'.format(last_proof_of_work, x, hashed_proof))
    return x

## running it
finally we invoke our `create_new_block` method. make sure to replace the argument with your name so we can see who mined the block! the `except` clause deals with the web server returning an error - if the proof of work is not valid it will appear here.

In [22]:
try:
    new_block = create_new_block('<YOUR NAME HERE>')
    response = post_block(new_block)
    print('___success!___')
    pretty_print(json.loads(response))
# if the request is incorrect, urllib will throw a HTTPError. get the details
except urllib.error.HTTPError as error:
    print('___request failed!___')
    pretty_print(json.loads(error.read().decode()))

hash of 5347 and 5137 is d7f5b3e943c9241aa0fb52796280821b81c61122bfc11f3edf4fe1a4b92ff000
{
    "blocks": [
        {
            "data": "{\"name\": \"jumblesale\", \"value\": 100}",
            "previous_hash": null,
            "timestamp": 1524343126.731536,
            "index": 0,
            "proof": 0
        },
        {
            "data": "{\"name\": \"bob\", \"value\": 1}",
            "previous_hash": "4794b4a613f7624f0efd41554294dcff030233483c6d2e7b1d6b74a8de5304d5",
            "timestamp": 1524343126.744977,
            "index": 1,
            "proof": 5735
        },
        {
            "data": "{\"name\": \"alice\", \"value\": 1}",
            "previous_hash": "62238b6b6e869ca5b20f63ca046f0e50ec32145d49f22e9699c6369da282593b",
            "timestamp": 1524343126.746572,
            "index": 2,
            "proof": 626
        },
        {
            "data": "{\"name\": \"<YOUR NAME HERE>\", \"value\": 1}",
            "previous_hash": "82a2bf94ce44c179df7af31ea6502e